In [ ]:
# Import the necessary modules
from pyspark.sql.functions import *
from ultis import *
from datetime import datetime
from decouple import config

spark = create_spark()

In [ ]:
spark.sql('''
SELECT
    A.DATE AS RUN_DATE,
    COUNT(*) AS TICKET_10D,
    FROM_UTC_TIMESTAMP(NOW(), 'GMT+7') AS LAST_EXECUTED_TIME
FROM delta.`s3a://silver/CALENDAR_DIM` A
LEFT JOIN delta.`s3a://silver/USER_RATING_FACT` B ON A.DATE = FROM_UTC_TIMESTAMP(B.CREATED_AT, 'GMT+7')
WHERE FROM_UTC_TIMESTAMP(B.CREATED_AT, 'GMT+7') >= DATE_ADD(A.DATE, -10)
GROUP BY A.DATE
''').write.format('delta').mode('overwrite').save('s3a://gold/USER_RATING_10D')

In [ ]:
spark.stop()